<a href="https://colab.research.google.com/github/ismoil27/jaydariGPT/blob/main/jaydari_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch bitsandbytes datasets peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.2 MB/s eta 0:00:00


In [20]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM, TrainingArguments
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

In [3]:
model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_id)

# print('Vocab size:', tokenizer.vocab_size)
# print('Special tokens:', tokenizer.special_tokens_map)

# quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type='nf4'
)

bnb_config

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map='auto', # GPU, CPU
    # dtype=torch.bfloat16
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
# Before Fine-tuning
prompt = "Explain what a tokenizer is?"
# prompt = "A tokenizer is a tool in natural language processing that"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device) # GPU, CPU

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.7
    )

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

# print(model)
first_block = model.model.layers[0]
print('first_block:', first_block)
print('=======')
print(first_block.self_attn)
print('=======')
print(model.config)



Explain what a tokenizer is?
first_block: LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
    (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
    (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
    (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
  )
  (mlp): LlamaMLP(
    (gate_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
    (up_proj): Linear4bit(in_features=2048, out_features=5632, bias=False)
    (down_proj): Linear4bit(in_features=5632, out_features=2048, bias=False)
    (act_fn): SiLUActivation()
  )
  (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
)
LlamaAttention(
  (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
  (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
  (v_proj): Linear4bit(in_features=2048, out_feat

In [5]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

total_params = count_parameters(model)
print(f"Total parameters (including frozen 4-bit): {total_params:,}")


Total parameters (including frozen 4-bit): 615,606,272


## datasets library | load_dataset
* instruction tuning

In [21]:
dataset = load_dataset("yahma/alpaca-cleaned", split="train")
dataset
dataset[1]

{'output': 'The three primary colors are red, blue, and yellow. These colors are called primary because they cannot be created by mixing other colors and all other colors can be made by combining them in various proportions. In the additive color system, used for light, the primary colors are red, green, and blue (RGB).',
 'input': '',
 'instruction': 'What are the three primary colors?'}

In [7]:
def generate_prompt(example):
  instruction = example['instruction']
  input_text = example['input']
  output_text = example['output']

  if input_text:
    return(
        "### Instruction:\n"
        f"{instruction}\n\n"
        "### Input:\n"
        f"{input_text}\n\n"
        "### Response:\n"
        f"{output_text}"
    )
  else:
    return(
       "### Instruction:\n"
       f"{instruction}\n\n"
       "### Response:\n"
       f"{output_text}"
    )

# generate_prompt(dataset[0])

def formatting_func(example):
  return {'text': generate_prompt(example)}

dataset = dataset.map(formatting_func)


Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [8]:
dataset[0]['text']

'### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.'

In [9]:
dataset = dataset.select(range(7000))

In [10]:
dataset = dataset.shuffle(seed=42)

In [11]:
# [7, 3, 2, 8, 5, 6, 9, 4, 0, 1]
# [7, 3, 2, 8, 5, 6, 9, 4, 0, 1] EXACT SAME ORDER

In [12]:
# [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# [10, 231, 342, 3453, 3464, 5123, 6456, 7, 8, 9]


In [13]:
dataset

Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 7000
})

In [14]:
# Full Fine-tuning  =>
# Cheap Fine-tuning =>
# PEFT => Parameter Efficent Fine Tuning
# OOM => Out of Memory

In [22]:
lora_config = LoraConfig(
    r=8, # rank
    lora_alpha=16,
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
)

In [16]:
model = get_peft_model(model, lora_config)

In [17]:
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [18]:
# QLoRa
# LoRa